In [1]:
from dotenv import load_dotenv
from metagame import TradingClient
import os

load_dotenv()

client = TradingClient(
    api_url=os.environ["API_URL"],
    jwt=os.environ["JWT"],
    act_as=int(os.environ["ACT_AS"]),
)

In [2]:
import pandas as pd

def get_market_trades(market_name: str):
    state = client.state()
    market_id = state.market_name_to_id.get(market_name)    
    if market_id is None:
        return pd.DataFrame()

    if not state.markets[market_id].hasFullTradeHistory:
        client.get_full_trade_history(market_id)
        state = client.state()

    trades = state.markets[market_id].trades
    if not trades:
        return pd.DataFrame()
    df = pd.DataFrame(trades).set_index("id")
    df["buyer"] = df["buyer_id"].map(lambda x: state.accounts[x].name)
    df["seller"] = df["seller_id"].map(lambda x: state.accounts[x].name)
    return df.drop(columns=["market_id", "buyer_id", "seller_id"])


In [3]:
def get_market_positions(market_name: str):
    df = get_market_trades(market_name)
    if df.empty:
        return pd.DataFrame()
    return pd.concat([
        df.groupby("buyer")["size"].sum(),
        -df.groupby("seller")["size"].sum()
    ]).groupby(level=0).sum()

def get_multiple_market_positions(market_names: list[str]):
    return pd.DataFrame({market: get_market_positions(market) for market in market_names}).fillna(0)

In [ ]:
import ipywidgets as widgets

def get_dice(refresh):
    markets = ["min_dice_c", "max_dice_c", "avg_dice_c"]
    return get_multiple_market_positions(markets)

widgets.interact(get_dice, refresh=widgets.Checkbox(description="Refresh Data"))